In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PCA import PCA_Component

data_1 = pd.read_csv("data/BATADAL_dataset03.csv")
data_2 = pd.read_csv("data/BATADAL_dataset04.csv", delimiter=r",\s{0,1}")
test_data = pd.read_csv("data/BATADAL_test_dataset.csv")

data_1["DATETIME"] = pd.to_datetime(data_1.DATETIME)
data_2["DATETIME"] = pd.to_datetime(data_2.DATETIME)
test_data["DATETIME"] = pd.to_datetime(test_data.DATETIME)
data_2['ATT_FLAG'] = data_2["ATT_FLAG"].replace(-999, 0)    # replace -999 with 0 for convenience

signals = ['L_T1', 'L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7',
       'F_PU1', 'S_PU1', 'F_PU2', 'S_PU2', 'F_PU3', 'S_PU3', 'F_PU4', 'S_PU4',
       'F_PU5', 'S_PU5', 'F_PU6', 'S_PU6', 'F_PU7', 'S_PU7', 'F_PU8', 'S_PU8',
       'F_PU9', 'S_PU9', 'F_PU10', 'S_PU10', 'F_PU11', 'S_PU11', 'F_V2',
       'S_V2', 'P_J280', 'P_J269', 'P_J300', 'P_J256', 'P_J289', 'P_J415',
       'P_J302', 'P_J306', 'P_J307', 'P_J317', 'P_J14', 'P_J422']

[0 1]


### Make test data `ATT_FLAG` column as described in `Attacks_TestDataset.png`

*NOTE:* We will make the state of no-arrack as 0 instead of -999

In [20]:
import datetime
test_data['ATT_FLAG'] = pd.Series(np.zeros(test_data.shape[0])) # default 0 (no-attack)

# start setting values
attack_times = [
    [pd.Timestamp("2017-01-16T09"), pd.Timestamp("2017-01-19T06")],
    [pd.Timestamp("2017-01-30T08"), pd.Timestamp("2017-02-02T00")],
    [pd.Timestamp("2017-02-09T03"), pd.Timestamp("2017-02-10T09")],
    [pd.Timestamp("2017-02-12T01"), pd.Timestamp("2017-02-13T07")],
    [pd.Timestamp("2017-02-24T05"), pd.Timestamp("2017-02-28T08")],
    [pd.Timestamp("2017-03-10T14"), pd.Timestamp("2017-03-13T21")],
    [pd.Timestamp("2017-03-25T20"), pd.Timestamp("2017-03-27T01")]
]

# showing how distribution of 0s and 1s changes
print(test_data['ATT_FLAG'].value_counts(normalize=True))
for times in attack_times:
    indexes = np.where(test_data['DATETIME'].between(times[0], times[1], inclusive=True))[0]
    test_data['ATT_FLAG'][indexes] = 1 # set 1 (attack)
    print(test_data['ATT_FLAG'].value_counts(normalize=True))

# make train X and test X Y
X_train = data_1[signals].to_numpy()
X_test = test_data[signals].to_numpy()
Y_test = test_data['ATT_FLAG'].to_numpy()

print(X_train.shape)
print(X_test.shape)
print(Y_test.shape)

0.0    1.0
Name: ATT_FLAG, dtype: float64
0.0    0.966491
1.0    0.033509
Name: ATT_FLAG, dtype: float64
0.0    0.946865
1.0    0.053135
Name: ATT_FLAG, dtype: float64
0.0    0.946865
1.0    0.053135
Name: ATT_FLAG, dtype: float64
0.0    0.943035
1.0    0.056965
Name: ATT_FLAG, dtype: float64
0.0    0.895165
1.0    0.104835
Name: ATT_FLAG, dtype: float64
0.0    0.884634
1.0    0.115366
Name: ATT_FLAG, dtype: float64
0.0    0.870273
1.0    0.129727
Name: ATT_FLAG, dtype: float64
(8761, 43)
(2089, 43)
(2089,)


(2089, 43)
(2089,)
